In [1]:
import pandas as pd
import numpy as np
import random
import os

# implementing random seed for control
sd = 42
np.random.seed(sd)

# school names for consistency 
schools = ['Oakland Middle School',
    'Siegel Middle School',
    'Whitworth-Buchanan Middle School',
    'Christiana Middle School',
    'Smyrna Middle School',
    'Stewarts Creek Middle School',
    'Rockvale Middle School',
    'Rocky Fork Middle School',
    'Blackman Middle School',
    # 'Thurman Francis Arts Academy',
    'Rock Springs Middle School',
    'LaVergne Middle School'
]

# get event year
try:
    event_year = int(input('Enter 4-digit year of YouScience Event\n'))
except ValueError:
    event_year = 2022
print(f"Running for YS Career Fair {event_year}")

# preparing directories TEST needs to be update in fnc export_POS_rosters(), also export_room_assignments()
log_pathway = f'../Logs/{event_year}/QA_Checks'
if not os.path.exists(log_pathway):
    os.makedirs(log_pathway)


Running for YS Career Fair 2023


### Checking for Repeated Pathways

In [28]:
# log_name = f"{event_year}_QA_Repeated_Pathways_check"
# with open(f'../Logs/{event_year}/QA_Checks/{log_name}.txt','w') as logf:
    
issues_dict = {}

for school in schools:
    issues_dict[school] = {} # k: index, v: repeated pathway

    # read in all_students.csv
    p = f"../YouScienceData/Schedules/{event_year}/{school}/{school}_all_students.csv" 
    # p = f"../../../../Desktop/2023/{school}/{school}_all_students.csv"
    df = pd.read_csv(p).drop("Unnamed: 0", axis=1)
    
    # check row for repeats
    for i in range(len(df)):
        X = df.iloc[i]
        pathways = list(X[['B1','B2','B3','B4']])
        if len(set(pathways)) < 4:
            print(f'{school} - Problem at {X.id}')
            student = X.id
            # find repeated pathway
            for pos in set(pathways):
                pathways.remove(pos)
            issues_dict[school][student] = pathways[0]
    
    


### Checking for Changed Room Assignments

In [61]:
room_issues_dict = {}

for school in schools:
    room_issues_dict[school] = {} # k: index, v: repeated pathway

    # read in all_students.csv
    revised_p = f"../YouScienceData/Schedules/{event_year}/{school}/room_assignments_by_demand.csv" 
    og_p = f"../../../../Desktop/2023/{school}/room_assignments_by_demand.csv"
    revised_df = pd.read_csv(revised_p).drop("Unnamed: 0", axis=1)
    og_df = pd.read_csv(og_p).drop("Unnamed: 0", axis=1)
    
    for i in range(len(og_df)):
        X, Y = og_df.iloc[i], revised_df.iloc[i]
        if X.POS != Y.POS:
            changes = {
                "Original Room":og_df.loc[og_df.POS == Y.POS]['Room'].values[0],
                "New Room":Y.Room,
            }
            room_issues_dict[school][Y.POS] = changes


In [62]:
Problems = []
for school in list(room_issues_dict.keys()):
    if room_issues_dict[school] != {}:
        Problems.append(school)

Problems

[]

### Why...? 

I am confused as to why room assignment changed at all. To my understanding, nothing in the fix implemented should have changed the assignments. Let's check to see if the rooms were swapped for equally sized rooms. If so, just revert those POS to the original rooms. 

In [48]:
room_issues_dict[school]

{'Audio Production': {'Original Room': '807', 'New Room': '806'},
 'Audio Visual Production': {'Original Room': '808', 'New Room': '807'},
 'Networking Systems': {'Original Room': '806', 'New Room': '808'}}

In [59]:

for school in Problems:
    # read in planning file worksheet 
    planning_df = pd.read_excel('planning.xlsx', sheet_name=school)
    check = 0
    for pathway in list(room_issues_dict[school].keys()):
        assignments = room_issues_dict[school][pathway]
        og_room, changed_room = int(assignments['Original Room']), int(assignments['New Room'])
        og_cap = planning_df.loc[planning_df['MS Room Number'] == og_room]['Max Room Capacity'].values[0]
        ch_cap = planning_df.loc[planning_df['MS Room Number'] == changed_room]['Max Room Capacity'].values[0]
        # if capacities match, revert change
        if og_cap == ch_cap:
            continue
        else:
            check += 1
            print(f"{school} - {pathway} - Capacity mismatch in room assignments") 

    if check == 0:
        print(f"Reversion possible for {school}.")
    else:
        print(f'Need to check rosters sizes for mismatched pathways to determine if reversion is possible.')


Reversion possible for Stewarts Creek Middle School.
Reversion possible for Rockvale Middle School.


In [60]:
# Revert 
for school in Problems:
    og_p = f"../../../../Desktop/2023/{school}/room_assignments_by_demand.csv"
    og_df = pd.read_csv(og_p).drop("Unnamed: 0", axis=1)
    og_df.to_csv(f'../YouScienceData/Schedules/2023/{school}/room_assignments_by_demand.csv')